In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('Lebanese_Civil_War')
page.get_parse()

en.wikipedia.org (parse) Lebanese_Civil_War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
Lebanese Civil War (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Martyrs S...
  infobox: <dict(10)> conflict, partof, image, image_size, caption...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:L...
  pageid: 312905
  parsetree: <str(129430)> <root><template><title>short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Lebanese Civil War
  wikibase: Q208484
  wikidata_url: https://www.wikidata.org/wiki/Q208484
  wikitext: <str(107439)> {{short description|1975–1990 civil war ...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Lebanese_Civil_War",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Lebanese Civil War')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
14 July Revolution
17 May Agreement
17 October Revolution
1860 conflict in Lebanon
1935 Yazidi revolt
1935–1936 Iraqi Shia revolts
1943 Barzani revolt
1947–1949 Palestine war
1947–48 Civil War in Mandatory Palestine
1948 Arab–Israeli War
1948 Palestinian exodus
1949 Armistice Agreements
1953 Iranian coup d'état
1958 Lebanon crisis
1959 Mosul uprising
1963 Syrian coup d'état
1964 Hama riot
1966 Syrian coup d'état
1967 Palestinian exodus
1967 Six Day War
1968 Israeli raid on Lebanon
1973 Yom Kippur War
1974–75 Shatt al-Arab conflict
1976 Arab League summit (Riyadh)
1977 Shia uprising in Iraq
1978 South Lebanon conflict
1979 Qatif Uprising
1979–1980 Shia uprising in Iraq
1980 Turkish coup d'état
1982 Beirut bombing
1982 Iranian diplomats kidnapping
1982 Lebanon War
1982 kidnapping of Iranian diplomats
1983 Beirut barracks bombing
1983 Beirut barracks bombings
1983 United States embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing
1984 United States embassy 


Page 2
List of speakers of the Parliament of Lebanon
List of wars involving Israel
Litani River
Little Satan
Lod Airport massacre
London Conference of 1939
London Conference of 1946–1947
Ma'alot massacre
Ma'n dynasty
Maarouf Saad
Madrid Conference of 1991
Mafia
Mahmoud Ahmadinejad and Israel
Mahmud Barzanji revolts
Mahsum Korkmaz
Majid Shahriari
Malcolm H. Kerr
Mamluk Sultanate (Cairo)
Mandate for Syria and the Lebanon
Mandatory Palestine
Marada Brigade
Marada Movement
March 14 Alliance
March 2012 Gaza–Israel clashes
March 2017 Israel–Syria incident
March 8 Alliance
Marcia C. Inhorn
Marjayoun
Maronite
Martyrs' Square, Beirut
Marxist-Leninist
Masoud Alimohammadi
May 17 Agreement
May 2013 Rif Dimashq airstrikes
McMahon–Hussein Correspondence
Memory Box (film)
Menachem Begin
Metn
Michel Aoun
Middle Eastern theatre of World War I
Military of Lebanon
Mirdasid dynasty
Misgav Am hostage crisis
Mohammad Hussein Fadlallah
Mohsen Fakhrizadeh
Monte Melkonian
Moral authority
Morocco
Morrison–Grad

In [4]:
def get_arabic_name(english_phrase):
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


# def get_list_items(div):
#     s = re.sub('{{.*?}}', '', div)
#     s = re.sub('\'', '', s)
#     s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     s = re.sub('\n----', '<br>', s)
#     s = re.sub('\n\*', '<br>', s)
#     s = re.sub('<br />', '<br>', s)
#     s = re.sub('<br/>', '<br>', s)
#     s = re.sub('</ref>|ref||</ref>', '', s)
#     s = re.sub('</ref>', '', s)
#     s = re.sub('ref', '', s)
# #     s = re.sub('|', '', s)
#     s = s.translate({ord('|'): '<br>'})
# #     if '<br>' in s:
#     s = s.split("<br>")
# #     else:
# #         s = s.split("\|")
#     s = [e.strip() for e in s if e.strip() != '']
#     s = [remove_file_instances(e).strip() for e in s]
#     s = [e for e in s if not to_delete(e)]
#     s = list(set(s)) # keep unique entries
#     return s


# def get_arabic_name(english_phrase):
#     http = urllib3.PoolManager()
#     url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
#     response = http.request('GET', url)

#     # get languages and links
#     soup = BeautifulSoup(response.data)
#     links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

#     for language, link in links:
#         if language == 'ar':
#             response = http.request('GET', link)
#             soup = BeautifulSoup(response.data)
# #             print(language, soup.title.text, link)
            
#             arabic_phrase = soup.title.text
#             arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
#             return arabic_phrase, link
#         else:
#             return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

# def remove_file_instances(s):
#     if '[[' in s and ']]' in s and 'File' in s:
#         ind2 = s.rfind(']')
#         return s[ind2+1:]
#     elif '[[' in s and ']]' in s and 'Image' in s:
#         ind2 = s.rfind(']')
#         return s[ind2+1:]
#     else:
#         return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s 

# def get_list_items(div):
#     s = re.sub('{{.*?}}', '', div)
#     s = re.sub('\'', '', s)
#     s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     s = re.sub('\n----', '<br>', s)
#     s = re.sub('\n\*', '<br>', s)
#     s = re.sub('<br />', '<br>', s)
#     s = re.sub('<br/>', '<br>', s)
#     s = re.sub('</ref>|ref||</ref>', '', s)
#     s = re.sub('</ref>', '', s)
#     s = re.sub('ref', '', s)
# #     s = re.sub('|', '', s)
#     s = s.translate({ord('|'): '<br>'})
# #     if '<br>' in s:
#     s = s.split("<br>")
# #     else:
# #         s = s.split("\|")
#     s = [e.strip() for e in s if e.strip() != '']
#     s = [remove_file_instances(e).strip() for e in s]
#     s = [e for e in s if not to_delete(e)]
#     s = list(set(s)) # keep unique entries
#     print('s is now: ', s)
#     new_s = []
#     for e in s:
#         if ',' in e:
#             splitted_e = e.split(',')
#             for ee in splitted_e:
#                 new_s.append(ee)
#         else:
#             new_s.append(e)
#     print('s is now after: ', new_s)
#     return new_s

In [5]:
# page = wptools.page('Hamas')
# page.get_parse(show=False)
# print('????????????????????????????????????????????????????????????????????????????????????')
# print(page.data['infobox']['ideology'])
# # print(page.data['infobox']['name'])


# name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
# ideology = get_list_items(page.data['infobox']['ideology'])
# position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

# print('-----------------')
# print('name: ', name)
# # print('founders: ', founders)
# print('ideology: ', ideology)
# count=0
# for i in ideology:
#     print(count, i)
#     count+=1
# print('position: ', position)

In [6]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [ ]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    if isinstance(name, list):
                        if len(name) > 1:
                            for n in name:
                                print('name before: ', n)
                                name = [n]
                                break
#                             name = list(name[0])
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 14 July Revolution
en.wikipedia.org (imageinfo) File:Abd al-Karim Qasim & Abd al-Sal...
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 17 October Revolution
en.wikipedia.org (imageinfo) File:Beirut Protests 2019.jpg
en.wikipedia.org (parse) 1860 conflict in Lebanon
en.wikipedia.org (imageinfo) File:DamasChristianQuarter1860.jpg
en.wikipedia.org (parse) 1935 Yazidi revolt
en.wikipedia.org (parse) 1935–1936 Iraqi Shia revolts
en.wikipedia.org (parse) 1943 Barzani revolt
en.wikipedia.org (parse) 1947–1949 Palestine war
en.wikipedia.org (parse) 1947–48 Civil War in Mandatory Palestine
en.wikipedia.org (imageinfo) File:Katamon.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1953 Iranian coup d'état
en.wikipedia.org (imageinfo) File:Operationajax.jpg
en.

-----------------
name:  ['Fatah – The Revolutionary Council']
founders:  
ideology:  ['Secularism', 'Anti-Zionism', 'Palestinian nationalism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/Fatah – The Revolutionary Council
name:  Fatah – The Revolutionary Council
name ar:  فتح - المجلس الثوري (مترجمه)
name ar link:  -1
END
START
ideology:  Secularism
ideology ar:  علمانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85%D8%A7%D9%86%D9%8A%D8%A9
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B5%D9%87%D9%8A%D9%88%D9%86%D9%8A%D8%A9
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Abu Nidal Organization internal executions
en.wikipedia.org (parse) Achaemenid Phoenicia
en.wikipedia.org (parse) Aden Emergency
en.wikipedia.org (imageinfo) File:South Yemen in its region.svg
en.wikipedia.org (parse) Administrative divisions of Lebanon
en.wikipedia.org (parse) Adwan Rebellion
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Agriculture in Lebanon
en.wikipedia.org (parse) Ahmed Barzani revolt
en.wikipedia.org (imageinfo) File:3.7inchHowitzerBarzanOperations...
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Akkar
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades


name before:  lang
name after:  ['lang']
-----------------
name:  ['lang']
founders:  
ideology:  ['Socialism', 'Anti-Zionism', 'Palestinian nationalism']
position:  -
START
name:  lang
name ar:  لانغ (توضيح) 
name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A7%D9%86%D8%BA_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
START
ideology:  Socialism
ideology ar:  اشتراكية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B5%D9%87%D9%8A%D9%88%D9%86%D9%8A%D8%A9
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Al-Mourabitoun


...

en.wikipedia.org (parse) Souha Bechara
en.wikipedia.org (imageinfo) File:Soha Bechara.jpg
en.wikipedia.org (parse) South Lebanon Army
en.wikipedia.org (imageinfo) File:Flag of the South Lebanon Army.png
en.wikipedia.org (parse) South Lebanon conflict (1985–2000)
en.wikipedia.org (imageinfo) File:Military post birkat hukban sou...
en.wikipedia.org (parse) South Yemen
en.wikipedia.org (parse) South Yemen Civil War
en.wikipedia.org (imageinfo) File:South Yemen.png
en.wikipedia.org (parse) South Yemen insurgency
en.wikipedia.org (imageinfo) File:South Yemen.png
en.wikipedia.org (parse) Southern Lebanon
en.wikipedia.org (parse) Soviet Union
en.wikipedia.org (parse) Sport in Lebanon
en.wikipedia.org (parse) Stuxnet
en.wikipedia.org (parse) Subhi al-Tufayli
en.wikipedia.org (imageinfo) File:Subhi al-Tufayli.jpg
en.wikipedia.org (parse) Sudan
en.wikipedia.org (parse) Suez Crisis
en.wikipedia.org (imageinfo) File:Tanks Destroyed Sinai.jpg
en.wikipedia.org (parse) Suleiman Franjieh
en.wikipedia.

[[Fares al-Saad]]
[[Antoun Saadeh]]
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Fares al-Saad,Antoun Saadeh,
ideology:  ['Syrian irredentism', 'Anti-Zionism', 'officially denied)', 'Anti-communism Page needed', 'ubl', 'Pan-Syrianism', 'Syrian nationalism', 'Economic populism', 'Antisemitism', 'August 2021', 'Secularism', 'officially denied) collapsible list', 'Page needed', 'Social nationalism']
position:  ['Syncretic', 'Left-wing collapsible list']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Syrian irredentism
ideology:  Syrian irredentism
ideology ar:  الوحدوية السورية (مترجمه)
ideo ar link:  -1
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ide

en.wikipedia.org (parse) Syrian civil war
en.wikipedia.org (parse) Syrian civil war spillover in Lebanon
en.wikipedia.org (imageinfo) File:Lebanese army on Syria Street.jpg
en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstrators.jpg
en.wikipedia.org (parse) Syria–Lebanon campaign
en.wikipedia.org (imageinfo) File:AUSTRALIAN FORCES IN LEBANON, 1...
en.wikipedia.org (parse) Ta'if
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Taif Agreement
en.wikipedia.org (parse) Tawhid Movement


name before:  حركة التوحيد الإسلامي
name after:  ['حركة التوحيد الإسلامي']
-----------------
name:  ['حركة التوحيد الإسلامي']
founders:  
ideology:  ['Islamism']
position:  -
START
Cannot translate phrase `حركة التوحيد الإسلامي` to arabic
END
START
ideology:  Islamism
ideology ar:  إسلاموية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A5%D8%B3%D9%84%D8%A7%D9%85%D9%88%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Tehran (TV series)
en.wikipedia.org (imageinfo) File:Tehran TV poster.jpg
en.wikipedia.org (parse) Tel al-Zaatar massacre
en.wikipedia.org (parse) Telecommunications in Lebanon
en.wikipedia.org (parse) Television in Lebanon
en.wikipedia.org (parse) Terrorism in Egypt
en.wikipedia.org (parse) Terrorism in Saudi Arabia
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) The Fateful Triangle: The United States,...
en.wikipedia.org (imageinfo) File:The Fateful Triangle.jpg
en.wikipedia.org (parse) The Human League
en.wikipedia.org (imageinfo) File:The Human League 2007.jpg
en.wikipedia.org (parse) The Lebanon (song)
en.wikipedia.org (imageinfo) File:LebanonHL.jpg
en.wikipedia.org (parse) The Vortices of Wrath (Lebanon 1977)
en.wikipedia.org (parse) Tigers Militia
en.wikipedia.org (parse) Tigers Militia (Lebanon)
en.wikipedia.org (parse) Timeline of Lebanese history
en.wikipedia.org (parse) Timeline of terrorism in Egypt (2013–present)
en.wikipe

In [ ]:
df_political_party_en

In [ ]:
df_political_party_ar

In [ ]:
datasets_dir = 'datasets_updated/'

In [ ]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'poltical_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'poltical_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [ ]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [ ]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        p_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

In [ ]:
df_politician_en

In [ ]:
df_politician_ar

In [ ]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')